### **5.8 - [Practica] Distributing your tasks with the Celery Executor**

#### **`worker_concurrency`**

Muy bien, en este momento usted debe tener su cluster ejecutando Airflow con el Celery Executor.  Si no es así, por favor revisa el video anterior. Escribiendo "docker ps" podemos ver que los contenedores siguen funcionando.  Ok, ahora vamos a ejecutar nuestro primer DAG distribuyendo sus tareas a los workers. Dado que sólo tenemos un worker, no debería ser realmente impresionante, pero no te preocupes, vamos a arreglar esto en un minuto.  Si recuerdas del video sobre el Local Executor hemos usado el DAG parallel_dag.  Bueno, aquí vamos a usarlo de nuevo.  Vamos a la interfaz de usuario de Airflow. Desde su navegador web vaya a localhost :8080.  Enter. Ya tenemos el DAG parallel_dag como esperábamos.  Abra una nueva pestaña y vaya a localhost:5555. Enter. Y aterrizamos en el Flower dashboard para monitorizar Celery y los workers donde debería tener solo un worker online como se muestra aquí.  Muy bien, antes de seguir adelante, vamos a ejecutar nuestro DAG. Vuelve a la interfaz de usuario de Airflow, activa el toggle del DAG parallel_dag y refresca la página hasta que veas las primeras tareas ejecutándose.  Por cierto, ya que el parámetro catchup se establece en True y la start_date está muy lejos en el pasado de la fecha actual, esto es perfecto para observar lo que está pasando con Celery y Airflow ya que vamos a tener un montón de DAGRuns para disparar. El DAG se está ejecutando, vamos al Flower dashboard. 

<center><img src="https://i.postimg.cc/bNCWHgVK/a829.png"></center>

Aquí, puedes ver que tenemos 8 columnas. De izquierda a derecha, está el nombre del worker, el estado que puede ser online u offline, el número de tareas activas, procesadas, fallidas, exitosas y reintentadas.  Por último, tenemos la columna de carga media (load average) con tres números correspondientes al número de procesos activos promediados en los últimos 1, 5 y 15 minutos respectivamente.  Estos números son importantes porque dan información sobre la carga del sistema (system load) de tu worker.  La pregunta es ¿cómo debes leer esos números?  Bueno, en general, ya que cada CPU core puede manejar un proceso a la vez, el sistema no está sobrecargado hasta que pasa de 1.0 por procesador lógico.  Por ejemplo, como tengo 8 cores en mi worker, mientras esos números se mantengan por debajo de 8, mi worker está bien.  Si usted tiene un dual core, mientras esos números se mantengan por debajo de 2, su worker está bien y así sucesivamente.  Normalmente, no pasa nada si la media de carga es superior a 1,0 por core en el último minuto.  Pero si obtienes más en los promedios de cinco y quince, eso podría indicar un problema.  Podrías encontrar las tareas intensivas en CPU y reenviarlas a un worker que tenga más CPUs, disminuir los parámetros de parallelism para ejecutar menos tareas o añadir más recursos a tu worker.  Bien, como puedes ver ya tenemos tareas procesadas y exitosas, pero si observas detenidamente la columna "active", el número de tareas activas nunca supera el 2.  ¿Por qué?  Porque en el vídeo sobre el Local Executor establecimos el parámetro dag_concurrency a 2 y así limitamos el número de tareas que se ejecutan en paralelo dentro del mismo DAG hasta 2. 

<center><img src="https://i.postimg.cc/0yh4hQsn/a830.png"></center>

Antes de cambiar este parámetro, me gustaría mostrarte alguna información interesante que puedes obtener de Flower.  Haz clic en el nombre del worker.  Desde ahí puedes acceder a diferente información como el pool de workers que te da el número máximo de procesos permitidos a ejecutar, 16 por defecto.  

<center><img src="https://i.postimg.cc/0Nm4vs5s/a831.png"></center>

El broker utilizado para extraer (pull) las tareas, que en nuestro caso es Redis.  

<center><img src="https://i.postimg.cc/KzDsKDgp/a832.png"></center>

Las colas (queues), desde donde se envían y extraen las tareas.  Veremos cómo crear una cola para ejecutar las tareas en un worker concreto. 

<center><img src="https://i.postimg.cc/gJWTkTr6/a833.png"></center>

Las tareas procesadas por el worker. 

<center><img src="https://i.postimg.cc/7PXWwX4f/a834.png"></center>

Los límites, donde se puede establecer un límite de tiempo antes de que el proceso de ejecución de una tarea se termina y un “rate limit” con el fin de definir el número máximo de tareas por minuto.  

<center><img src="https://i.postimg.cc/SRQ1cY6S/a835.png"></center>

Luego tenemos la configuración, donde puede obtener opciones de configuración del worker.  Siéntase libre de hacer clic en las opciones para aprender más sobre ellas.  

<center><img src="https://i.postimg.cc/QC64mNmp/a836.png"></center>

Por último, está la vista System donde puedes encontrar las estadísticas de uso del sistema

<center><img src="https://i.postimg.cc/xTCFCHjg/a837.png"></center>

Muy bien, dicho esto, vamos a cambiar el parámetro de parallelism para aumentar el número de tareas ejecutadas que se pueden ejecutar al mismo tiempo. Abramos nuestro editor de código.  Comprueba que estás en la carpeta airflow-materials/airflow-section-5 y abre airflow.cfg en la carpeta mnt/airflow. Busca el parámetro parallelism.  Como puede ver, está establecido en 3. Cámbielo por 16 para que podamos ejecutar hasta 16 tareas en paralelo en Airflow.  Justo debajo, establece el parámetro dag_concurrency a 16 también.  Ahora, dentro del mismo DAGRun podemos ejecutar hasta 16 tareas en paralelo. Guarda el archivo, y vuelve a tu terminal. 

<center><img src="https://i.postimg.cc/Hshv7xVm/a838.png"></center>

Escribe **`"docker-compose -f docker-compose-CeleryExecutor.yml down"`** para detener los contenedores docker e inícialos de nuevo con **`"docker-compose -f docker-compose- CeleryExecutor.yml up -d"`**.  Comprobemos que todo funciona bien escribiendo **`"docker ps"`**. Perfecto. 

Volvamos al navegador web y refresquemos la interfaz de usuario de Airflow hasta que vuelva a estar arriba. Haz lo mismo con el Flower dashboard.  Perfecto. Vuelve a la interfaz de Airflow, activa el toggle del DAG parallel_dag y refresca la página hasta que veas las primeras tareas ejecutándose.

<center><img src="https://i.postimg.cc/mrxd85Y6/a839.png"></center>

Ahora, desde el Flower dashboard si echas un vistazo a la columna “active” verás que hay más de 2 tareas activas.  

<center><img src="https://i.postimg.cc/pL5GQHFF/a840.png"></center>

Obviamente el número de tareas activas al mismo tiempo depende de las dependencias de tu DAG y si múltiples DAGRuns se están ejecutando al mismo tiempo.  Te dejo esto como ejercicio, pero si detienes los contenedores docker, establece un valor mayor que 1 como 16 al parámetro max_active_runs_per_dag y reinicia los contenedores docker como hicimos nosotros. Después de programar (scheduling) tu parallel_dag, tendrás hasta 16 tareas activas al mismo tiempo. No dudes en hacerlo. 

Muy bien, el último parámetro que me gustaría mostrarte es "**`worker_concurrency`**".  De vuelta al editor, encuentra el parámetro "**`worker_concurrency`**". **Este parámetro determina cuántas tareas puede procesar un solo worker. Por defecto se establece en 16**.  

<center><img src="https://i.postimg.cc/jjzM94vq/a841.png"></center>

**`Así, si tienes 4 workers ejecutándose a un`** **worker_concurrency** **`de 16, podrías procesar hasta 64 tareas a la vez. Sin embargo, si el parámetro`** **parallelism** **`se establece en 16, entonces sólo 16 tareas en realidad se ejecutan en paralelo y no 64. Lo mismo con el parámetro dag_concurrency. Si lo estableces en 5, sólo 5 tareas podrán ejecutarse en paralelo para un determinado DAGRun en tus workers`**. Para el parámetro "worker_concurrency", más procesos son usualmente lo mejor, pero hay un punto de corte donde agregar más procesos afecta el rendimiento de manera negativa. Necesitas hacer tus experimentos para encontrar el número que funcione mejor para ti, ya que esto varía según la aplicación, la carga de trabajo (workload), los tiempos de ejecución (runtimes) de las tareas y otros factores.  Así que, para darte una mejor idea de todos estos parámetros, echemos un vistazo al siguiente esquema que hice para ti. 

___

#### **`Esquema Airflow instance`**

<center><img src="https://i.postimg.cc/Pxy2ffBB/a842.png"></center>

En este esquema he ordenado los parámetros según su prioridad.  Así que aquí, hasta 32 tareas se ejecutarán en paralelo para la instancia de Airflow con **`parallelism`** igual a 32. Entonces, como **`max_active_run_per_dag`** es igual a 2, sólo dos DAGRuns del mismo DAG podrán ejecutarse en cualquier momento. A continuación, **`dag_concurrency`** se establece en 4, lo que significa que se pueden ejecutar hasta 4 tareas en paralelo dentro del mismo DAGRun, incluso si el parámetro **`worker_concurrency`** se establece en 16.  

Ahora, si cambiamos el valor de **`dag_concurrency`** a 16, 32 tareas podrán ejecutarse de forma concurrente. 16 en DAGRun #1 y 16 en DAGRun #2.  

<center><img src="https://i.postimg.cc/cCJTvyW6/a843.png"></center>

No dudes en jugar con estos parámetros ajustándolos y reiniciando los contenedores docker como lo hicimos para comprender plenamente el impacto de cada parámetro.  De acuerdo.  En realidad, vamos a disminuir el worker_concurrency a 1 para ver lo que obtenemos.  Vuelve al editor, establece el parámetro en 1 y guarda el archivo. Vuelve a tu terminal, y reinicia los contenedores docker con los comandos que usamos antes.  

<center><img src="https://i.postimg.cc/BbwpDYZp/a844.png"></center>

Ok, ahora que hemos reiniciado Airflow volvamos a la interfaz de usuario.  Activa el toggle del DAG parallel_dag para programarlo y actualiza la página. Bien, el DAG está funcionando, podemos ir al Flower Dashboard.  Si te fijas bien en la columna "active", puedes ver que, ahora sólo hay una tarea activa en cada momento ya que hemos limitado el número de procesos por worker a 1. 

<center><img src="https://i.postimg.cc/76wKyJQ3/a845.png"></center>

Como sólo tenemos un worker, sólo se puede ejecutar una tarea.  Si seguimos la lógica, añadiendo un nuevo worker deberíamos poder ejecutar dos tareas en paralelo, ¿no?  Eso es exactamente lo que vamos a comprobar en el siguiente vídeo. Por ahora, vuelve a tu terminal, detén los contenedores docker escribiendo **`"docker-compose -f docker-compose-CeleryExecutor.yml down"`**.  